## Lagrange Multipliers 
Note, f (optimizing functions) and g (constraints) are both multivariate polynomials

#### Assumptions
- This notebook can only work with one constraint (I have created another notebook that is generalized, do check that)
- It is possible that some variable is "free", i.e., its value can be taken any number in the field. I have not considered that as it makes symbolic root solving for other variables difficult.
- Technically, I am calculating the optimum value only at critical points. If the region under constraints is not bounded by above (below) then a maximum (minimum) doesn't exist. I am not checking the boundedness of region here

In [1]:
import numpy as np

In [2]:
set_random_seed(1)

In [3]:
n = 3

In [4]:
F = RR # Field 

In [5]:
variables = ['x'+str(i) for i in range(n)]; variables

['x0', 'x1', 'x2']

In [6]:
P = PolynomialRing(F, variables, order='lex'); P

Multivariate Polynomial Ring in x0, x1, x2 over Real Field with 53 bits of precision

In [7]:
x = P.gens(); x

(x0, x1, x2)

In [8]:
f = P.random_element(); f

0.964652347918136*x0^2 + 0.364600307131730*x0*x2 - 0.628246299697287*x0 - 0.308688596474854*x2^2 + 0.541454003184010

In [9]:
g = P.random_element(); g

-0.497866351436908*x0^2 + 0.816703331508811*x0*x2 - 0.947197913825740*x0 + 0.269469347982694*x1*x2 - 0.252736150629489*x2^2

In [10]:
variables.insert(0,'L'); variables

['L', 'x0', 'x1', 'x2']

In [11]:
Pnew = PolynomialRing(F, variables, order='lex'); Pnew

Multivariate Polynomial Ring in L, x0, x1, x2 over Real Field with 53 bits of precision

In [12]:
x = Pnew.gens(); x

(L, x0, x1, x2)

In [13]:
f = Pnew(f)
g = Pnew(g)

In [14]:
f_gradient = f.gradient(); f_gradient

[0,
 1.92930469583627*x0 + 0.364600307131730*x2 - 0.628246299697287,
 0,
 0.364600307131730*x0 - 0.617377192949707*x2]

In [15]:
g_gradient = g.gradient(); g_gradient

[0,
 -0.995732702873815*x0 + 0.816703331508811*x2 - 0.947197913825740,
 0.269469347982694*x2,
 0.816703331508811*x0 + 0.269469347982694*x1 - 0.505472301258978*x2]

In [16]:
L = Pnew.gens()[0]; L

L

In [17]:
equations = [f_gradient[i] - L * g_gradient[i] for i in range(1, n+1)]
equations.append(g); equations

[0.995732702873815*L*x0 - 0.816703331508811*L*x2 + 0.947197913825740*L + 1.92930469583627*x0 + 0.364600307131730*x2 - 0.628246299697287,
 -0.269469347982694*L*x2,
 -0.816703331508811*L*x0 - 0.269469347982694*L*x1 + 0.505472301258978*L*x2 + 0.364600307131730*x0 - 0.617377192949707*x2,
 -0.497866351436908*x0^2 + 0.816703331508811*x0*x2 - 0.947197913825740*x0 + 0.269469347982694*x1*x2 - 0.252736150629489*x2^2]

In [18]:
I = Ideal(equations); I

Ideal (0.995732702873815*L*x0 - 0.816703331508811*L*x2 + 0.947197913825740*L + 1.92930469583627*x0 + 0.364600307131730*x2 - 0.628246299697287, -0.269469347982694*L*x2, -0.816703331508811*L*x0 - 0.269469347982694*L*x1 + 0.505472301258978*L*x2 + 0.364600307131730*x0 - 0.617377192949707*x2, -0.497866351436908*x0^2 + 0.816703331508811*x0*x2 - 0.947197913825740*x0 + 0.269469347982694*x1*x2 - 0.252736150629489*x2^2) of Multivariate Polynomial Ring in L, x0, x1, x2 over Real Field with 53 bits of precision

In [19]:
gb = I.groebner_basis(); gb

[L + 0.821300000000000*x1 - 25.3300000000000*x2 - 0.663300000000000, x0 + 0.300400000000000*x1 - 12.3600000000000*x2, x1^2 - 6.33300000000000*x1 + 6.76100000000000*x2, x1*x2 - 6.14300000000000*x2, x2^2 - 0.173000000000000*x2]

In [20]:
gb_matrix = [list(gb[i].dict().items()) for i in range(len(gb))]; gb_matrix

[[((1, 0, 0, 0), 1.00000000000000),
  ((0, 0, 1, 0), 0.821300000000000),
  ((0, 0, 0, 1), -25.3300000000000),
  ((0, 0, 0, 0), -0.663300000000000)],
 [((0, 1, 0, 0), 1.00000000000000),
  ((0, 0, 1, 0), 0.300400000000000),
  ((0, 0, 0, 1), -12.3600000000000)],
 [((0, 0, 2, 0), 1.00000000000000),
  ((0, 0, 1, 0), -6.33300000000000),
  ((0, 0, 0, 1), 6.76100000000000)],
 [((0, 0, 1, 1), 1.00000000000000), ((0, 0, 0, 1), -6.14300000000000)],
 [((0, 0, 0, 2), 1.00000000000000), ((0, 0, 0, 1), -0.173000000000000)]]

In [21]:
variables_present = []
for polynomial in gb_matrix:
    variables = []
    for term in polynomial:
        for i, variable in enumerate(term[0]):
            if variable != 0 and i not in variables:
                variables.append(i)
    variables_present.append(variables)
variables_present

[[0, 2, 3], [1, 2, 3], [2, 3], [2, 3], [3]]

In [22]:
all_values = []

In [23]:
def backtrack_multivariate(values, gb, gb_matrix, variables_present, current_index, previous_index, all_values):
    i = len(values)
    if i > n:
        all_values.append(values)
        return
    
    previous_index = current_index
    current_index = current_index - 1
    to_eliminate = n-i-1
    while current_index>=0:
            temp_bool = true
            for j in variables_present[current_index]:
                temp_bool = temp_bool and (j>to_eliminate)
            if temp_bool is False:
                current_index = min(current_index + 1, len(gb)-1)
                break
            else:
                current_index = current_index - 1
    current_index = max(0, current_index)
    if i > 0:    
        gb_new = []
        for j in range(len(gb)):
            gb_new.append(gb[j].subs({x[n+1-i]:values[0]}))
        gb_matrix_new = [list(gb_new[z].dict().items()) for z in range(len(gb_new))];
    else: 
        gb_new = gb
        gb_matrix_new = gb_matrix
       
    all_roots = []
    for k in range(current_index, previous_index):
        roots = []
        try:
            factors = gb_new[k].factor()
        except:
            continue
        for fact in factors:
            polynomial = fact[0].dict()
            fac = list(polynomial.items())
            if len(fac) == 1:
                if 0 not in roots:
                    roots.append(0)
                
            if len(fac) == 2:
                if sum(fac[1][0]) < 2:
                    value = -fac[0][1]/fac[1][1]
                    if value not in roots:
                        roots.append(value)
        all_roots.append(roots)
    
    if (len(all_roots)) == 0:
        print("Could not proceed with values ", values, "returning")
        print("Possible reason: No solution or a variable can take infinite values")
        return
    common_roots = all_roots[0]
    for j in range(1,len(all_roots)):
        common_roots = set(common_roots).intersection(all_roots[j])
    if len(common_roots) == 0:
        try:
            common_roots.add(0)
        except:
            try:
                common_roots.append(0)
            except:
                return
    
    for root in common_roots:
        new = values
        new.insert(0,root)
        backtrack_multivariate(new.copy(), gb_new.copy(), gb_matrix_new.copy(), variables_present, current_index, previous_index, all_values)
        values = values[1:]
        

In [24]:
all_values = []

In [25]:
backtrack_multivariate([], gb, gb_matrix, variables_present, len(gb_matrix), 0, all_values)

In [26]:
minimum_value = (0,np.inf)
maximum_value = (0,-np.inf)
for i,values in enumerate(all_values):
    value_assignments = {}
    for j, value in enumerate(values):
        value_assignments[x[j]] = value
    f_value = f.subs(value_assignments)
    if f_value < minimum_value[1]:
        minimum_value = (i,f_value)
    if f_value > maximum_value[1]:
        maximum_value = (i,f_value)
    print(str(x), " is ", values, " with f", str(x[1:]), "=", f_value)
print("Minimum is at ", all_values[minimum_value[0]], " and the minimum value is ", minimum_value[1])
print("Maximum is at ", all_values[maximum_value[0]], " and the maximum value is ", maximum_value[1])

(L, x0, x1, x2)  is  [0.663300000000000, 0, 0, 0]  with f (x0, x1, x2) = 0.541454003184010
(L, x0, x1, x2)  is  [-4.53799290000000, -1.90243320000000, 6.33300000000000, 0]  with f (x0, x1, x2) = 5.22797063863078
(L, x0, x1, x2)  is  [5.04539000000000, 2.13828000000000, 0, 0.173000000000000]  with f (x0, x1, x2) = 3.73434596133129
Minimum is at  [0.663300000000000, 0, 0, 0]  and the minimum value is  0.541454003184010
Maximum is at  [-4.53799290000000, -1.90243320000000, 6.33300000000000, 0]  and the maximum value is  5.22797063863078
